In [1]:
# importing various module to be used in program
import requests
import os
import zipfile
import openpyxl
import sqlite3
import statistics
import csv

#Creating a staging subdirectory for cleansing of exxtratced data
staging_dir_name="staging"
os.mkdir(staging_dir_name)

#Method to unzipping the zip file
def unzip_file(zip_file_name):
    zf=open(zip_file_name,"wb")
    #Validating the content length
    if r.headers['Content-Length']==str(zf.write(r.content)):
        zf.close()
        z=zipfile.ZipFile(zip_file_name,"r")
        #Extracting the zip file in staging directory
        z.extractall(staging_dir_name)
        #Converting the file from windows encoding(cp1252) to UTF-8 enconding in fix file
        for f in os.listdir(staging_dir_name):
            #Not considering the corrupt file
             if f.endswith('.csv') and f!='FY2015_Percent_Change_in_Medicare_Payments.csv':
                file_name = os.path.join(staging_dir_name,f)
                # read in a file with code page cp1252 (Windows English US)
                ifp = open(file_name, "rt", encoding='cp1252')
                input_data = ifp.read()
                ifp.close()
                ofp = open(file_name + ".fix","wt", encoding='utf-8')
                for c in input_data:
                #Condition for Null Values
                    if c != '\0':
                        ofp.write(c)
                ofp.close()
                
#method to derive hospital information on the basis of provider id                
def drv_hospital(m=4806):
    wb_hrfs = openpyxl.load_workbook("hospital_ranking_focus_states.xlsx")
    hrfs_sheet_1_name = wb_hrfs.get_sheet_names()[0]
    hrfs_sheet_1 = wb_hrfs.get_sheet_by_name(hrfs_sheet_1_name)
    i = 2
    t=[]
    #loading the data generated from select statement to list
    while hrfs_sheet_1.cell(row = i, column = 1).value != None and i<=m+1:
        k=hrfs_sheet_1.cell(row = i,column = 1).value
        l=hrfs_sheet_1.cell(row = i,column = 2).value
        sql_str="Select hospital_name, city, state, county_name from hospital_general_information where provider_id =\""+k+"\""
        #Running select statement
        rows= c1.execute(sql_str)
        for row in rows:
            (n,b,c,d)=row
        t.append([l,k,n,b,c,d])
        i+=1
    return(t)

#Method to insert header of sheet
def sheet1_label(a):
    for i in range(len(a)):
        sheet_1_hr.cell(row = 1, column =i+1 , value=a[i])

#Method to insert header of sheet
def sheet2_label(a):
    for i in range(len(a)):
        sheet_2_hr.cell(row = 1, column =i+1 , value=a[i])

#method to add data in statewise sheet
def sql_extrt_cmd(sql_str):
    t=[]
    rows= c1.execute(sql_str)
    for row in rows:
        (a,b,c,d)=row
        t.append([a,b,c,d])
    return t
    
#Downloading the medicare data file in staging directory
if os.path.isdir(staging_dir_name) is True:
    url="https://data.medicare.gov/views/bg9k-emty/files/0a9879e0-3312-4719-a1db-39fd114890f1?content_type=application%2Fzip%3B%20charset%3Dbinary&filename=Hospital_Revised_Flatfiles.zip"
    r=requests.get(url)
    a=r.headers['Content-disposition'].find('=')
    zip_file_name=os.path.join(staging_dir_name,r.headers['Content-disposition'][(a+1):])
    #Calling unzip file function to extract csv files
    unzip_file(zip_file_name)
   
 #Connecting to database
conn = sqlite3.connect("medicare_hospital_compare.db")
c1 = conn.cursor()
# Checking with all files in the staging directory
for f in os.listdir(staging_dir_name):
     if f.endswith('.fix'):
            w=[]
            #Naming convention for table
            z=((((f[0:f.find(".")].lower()).replace(" ","_")).replace("-","_")).replace("%","pct")).replace("/","_")
            if z[0].isalpha():
                pass
            else:
                z="t_"+z
            k= os.path.join(staging_dir_name,f)
            #Reading the fixed file to extract data
            with open(k, mode="rt", newline='', encoding='utf-8') as x:
                reader = csv.reader(x)
                for row in reader:
                    if len(row)!=1:
                        w.append(row)
                    else:
                        pass
            #Executing sql statement for dropping table if already present
            sql_str = "drop table if exists " +z 
            c1.execute(sql_str)
            #Executing sql statement for creating table
            sql_str = "create table if not exists " +z
            sql_str_insy="insert into " +z 
            val=" Values("
            for i in w[0]:
                #naming convention for Columns
                q=((((i.lower()).replace(" ","_")).replace("-","_")).replace("%","pct")).replace("/","_")
                if q[0].isalpha():
                    pass
                else:
                    q="c_"+q
                #Structure for create and insert statement
                if w[0].index(i)==0:
                    sql_str=sql_str +"("+q+" text,"
                    sql_str_insy=sql_str_insy +"("+q+","
                    val=val+ "?,"
                elif w[0].index(i)==len(w[0])-1:
                    sql_str=sql_str +" "+q+" text)"
                    val=val+ "?)"
                    sql_str_insy=sql_str_insy +" "+q+")"+val
                else:
                    sql_str=sql_str +" "+q+" text,"
                    sql_str_insy=sql_str_insy +" "+q+","
                    val=val+"?,"
            #Executing create statement
            c1.execute(sql_str)
            b=[]
            for j in range(1,len(w)):
                p=tuple(w[j])
                b.append(p)
            #Executing insert statement
            c1.executemany(sql_str_insy,b)
            conn.commit()
            
 #Downloading hospital focus file from internet
url_2 = "http://kevincrook.com/utd/hospital_ranking_focus_states.xlsx"
r = requests.get(url_2)

#method to derive table as per the list provided from database
xf = open("hospital_ranking_focus_states.xlsx","wb")
if xf.write(r.content)==int(r.headers['Content-Length']):
    xf.close()
    wb_hr = openpyxl.Workbook()
    sheet_1_hr = wb_hr.create_sheet("Nationwide")
    wb_hr.remove_sheet(wb_hr.get_sheet_by_name("Sheet"))
    wb_hr.save("hospital_ranking.xlsx")
    #Column header for Excel sheets
    a=["Provider ID","Hospital Name","City","State","County"]
    sheet1_label(a)
    e=drv_hospital(101)
    i=2
    #Writing data into hospital ranking
    while i<=len(e):
        sheet_1_hr.cell(row = i, column = 1, value=e[i-2][1])
        sheet_1_hr.cell(row = i, column = 2, value=e[i-2][2])
        sheet_1_hr.cell(row = i, column = 3, value=e[i-2][3])
        sheet_1_hr.cell(row = i, column = 4, value=e[i-2][4])
        sheet_1_hr.cell(row = i, column = 5, value=e[i-2][5])
        i+=1
    wb_hr.save("hospital_ranking.xlsx")
   #Creating state wise sheet in hospital_ranking 
    wb_hrfs = openpyxl.load_workbook("hospital_ranking_focus_states.xlsx")
    hrfs_sheet_2_name = wb_hrfs.get_sheet_names()[1]
    hrfs_sheet_2 = wb_hrfs.get_sheet_by_name(hrfs_sheet_2_name)
    e=drv_hospital()
    p=[]
    i=1
    v=0
    #Validating with the hospital focus sheet
    while hrfs_sheet_2.cell(row = i, column = 1).value != None:
        sheet_2_hr = wb_hr.create_sheet(hrfs_sheet_2.cell(row = i,column = 1).value)
        sheet2_label(a)
        v=2
        #Creating sheet for each state
        for j in range(len(e)):
            if e[j][4]==hrfs_sheet_2.cell(row = i,column = 2).value and v<=101:
                #p.append(e[j])
                sheet_2_hr.cell(row = v, column = 1, value=e[j][1])
                sheet_2_hr.cell(row = v, column = 2, value=e[j][2])
                sheet_2_hr.cell(row = v, column = 3, value=e[j][3])
                sheet_2_hr.cell(row = v, column = 4, value=e[j][4])
                sheet_2_hr.cell(row = v, column = 5, value=e[j][5])
                v+=1   
        i += 1
    wb_hr.remove_sheet(wb_hr.get_sheet_by_name("State Name"))
    wb_hr.save("hospital_ranking.xlsx")
    
#Creating Nationwide in Measure statistics
wb_hr = openpyxl.Workbook()
sheet_1_hr = wb_hr.create_sheet("Nationwide")
wb_hr.remove_sheet(wb_hr.get_sheet_by_name("Sheet"))
#Headers for Measure statistics
a=["Measure ID","Measure Name","Minimum","Maximum","Average","Standard Deviation"]
sheet1_label(a)
wb_hr.save("measure_statistics.xlsx")
sql_str="Select state, measure_id, measure_name, score from timely_and_effective_care___hospital"
t=sql_extrt_cmd(sql_str)   
sql_str="Select distinct measure_id from timely_and_effective_care___hospital"
rows= c1.execute(sql_str)
g=[]
for row in rows:
    (o,)=row
    g.append(o.replace("\"",""))

v=2
#Calculating Statistic for nationwide
for i in range(len(g)):
    my_list=[]
    for j in t:
        if g[i]==j[1] and j[3].isnumeric():
            my_list.append(int(j[3]))
            s=j[2]
        else:
            pass
        #Valiating the length of list
    if len(my_list)!=0 and v<=len(g)+1:
        sheet_1_hr.cell(row = v, column = 1, value=g[i])
        sheet_1_hr.cell(row = v, column = 2, value=s)
        sheet_1_hr.cell(row = v, column = 3, value=min(my_list))
        sheet_1_hr.cell(row = v, column = 4, value=max(my_list))
        sheet_1_hr.cell(row = v, column = 5, value=statistics.mean(my_list))
        sheet_1_hr.cell(row = v, column = 6, value=statistics.pstdev(my_list))
        v+=1
        wb_hr.save("measure_statistics.xlsx")
    else:
        pass

#Opening the hospital focus list sheet 
wb_hrfs = openpyxl.load_workbook("hospital_ranking_focus_states.xlsx")
hrfs_sheet_2_name = wb_hrfs.get_sheet_names()[1]
hrfs_sheet_2 = wb_hrfs.get_sheet_by_name(hrfs_sheet_2_name)

#Creating statewise sheets in measure_statistic
i=2
while hrfs_sheet_2.cell(row = i, column = 1).value != None:
    x=0
    v=2
    sheet_2_hr = wb_hr.create_sheet(hrfs_sheet_2.cell(row = i,column = 1).value)
    sheet2_label(a)
    wb_hr.save("measure_statistics.xlsx")
    for x in range(len(g)):
        my_list=[]
        for j in t:
            if hrfs_sheet_2.cell(row = i, column = 2).value==j[0] and g[x]==j[1] and j[3].isnumeric() :
                my_list.append(int(j[3]))
                s=j[2]
            else:
                pass
        #Calculating statistics for each measure id
        if len(my_list)!=0 and v<=len(g)+1:
            sheet_2_hr.cell(row = v, column = 1, value=g[x])
            sheet_2_hr.cell(row = v, column = 2, value=s)
            sheet_2_hr.cell(row = v, column = 3, value=min(my_list))
            sheet_2_hr.cell(row = v, column = 4, value=max(my_list))
            sheet_2_hr.cell(row = v, column = 5, value=statistics.mean(my_list))
            sheet_2_hr.cell(row = v, column = 6, value=statistics.pstdev(my_list))
            v+=1
        else:
            pass
    i+=1
wb_hr.save("measure_statistics.xlsx")
    